<a href="https://colab.research.google.com/github/ragul-n/Analysis-of-Ashoka-University-Confession-page/blob/master/Update_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook updates the the following datasets:
1. sentiments.csv
2. opnions.csv
3. likes_and_comments.csv

**Instruction:** 
1. Set the ```last_update``` and ```today``` to appropriate date and run the Notebook




In [ ]:
import datetime 

last_update= datetime.datetime(2021, 8, 16)
today= datetime.datetime(2021,8,17)


In [ ]:
!pip install instaloader
!pip install --upgrade azure-cognitiveservices-vision-computervision
!pip install pillow
!pip install azure-ai-textanalytics==5.1.0

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image
import nltk
import random
import re
import IPython.display



from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time


nltk.download("vader_lexicon")

from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Downloding Posts

In [ ]:
import instaloader

from datetime import datetime
from itertools import dropwhile, takewhile


L = instaloader.Instaloader()

L.interactive_login("ragulnatarajan") 

profile = instaloader.Profile.from_username(L.context, "au.rants.23")

posts= profile.get_posts()
posts = list(posts)


In [ ]:

SINCE = last_update
UNTIL = today


new_posts= list(takewhile(lambda p: p.date > SINCE, dropwhile(lambda p: p.date > UNTIL, posts)))

count=0
length=len(new_posts)
info = display(str(count) + "/"+ str(length), display_id=True)

for post in new_posts:

    info = display(str(count) + "/"+ str(length), display_id=True)

    print(post.date)
    L.download_post(post, "instagram")
    count+=1



In [ ]:
image_list=[]

for file in os.listdir("/content/instagram"):
    if file.endswith(".jpg"):
        image_list.append(os.path.join("/content/instagram", file))


def reduce_key(fn):
    # you can change this according to your definition of "similar"
    return re.sub('_\d', '', fn)

from collections import defaultdict
# this holds the grouped filenames
group_dict = defaultdict(list)
for fn in image_list:
    key = reduce_key(fn)
    group_dict[key].append(fn)


image_list = list(group_dict.values())

len(image_list)

13

Extracting Texts

In [ ]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = "ab1b27f9f3fe48a6aac6b3de0224e94a"
endpoint = "https://ragul.cognitiveservices.azure.com/"


computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))



def read_image(read_image_path):

    # Open the image
    read_image = open(read_image_path, "rb")

    # Call API with image and raw response (allows you to get the operation location)
    read_response = computervision_client.read_in_stream(read_image, raw=True)
    # Get the operation location (URL with ID as last appendage)
    read_operation_location = read_response.headers["Operation-Location"]
    # Take the ID off and use to get results
    operation_id = read_operation_location.split("/")[-1]

    # Call the "GET" API and wait for the retrieval of the results
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status.lower () not in ['notstarted', 'running']:
            break
        
    # Print results, line by line
    string=""
    
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                string = string + " " + line.text
                #print(line.bounding_box)
    return string

In [ ]:
rants=[]

count=0
length= str(len(image_list))
info = display("No of files compleated: "+str(count) + "/"+ length, display_id=True)

for images in image_list:

    match = re.search(r'\d{4}-\d{2}-\d{2}', images[0])
    date = datetime.strptime(match.group(), '%Y-%m-%d').date()
    id = re.search(r'\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}', images[0]).group()
    

    rant=""
    for image in sorted(images):
        rant += read_image(image)

    rants.append({"id":id, "date":date, "rant":rant})

    count+=1
    info.update("No of files compleated: "+str(count) + "/"+ length)

      

'No of files compleated: 13/13'

## Text Analysis

In [ ]:
key = "384eb6ff06b9458bb15231adce28ba5f"
endpoint = "https://ragul-text.cognitiveservices.azure.com/"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()



def sentiment_analysis(rant, client):

    length=len(rant)
    if length  != 0:

        if length > 5120:
            rant=rant[:5120]

        documents = [rant]
        response = client.analyze_sentiment(documents=documents)[0]
        
        sentiment= {
            "sentiment":responce.sentiment,
            "positive":response.confidence_scores.positive,
            "neutral":response.confidence_scores.neutral,
            "negative":response.confidence_scores.negative,
        }

        return sentiment
    else:
        return {
            "positive":np.nan,
            "neutral":np.nan,
            "negative":np.nan,
        }


sentiments=[]

count=0
length= str(len(image_list))
info = display(str(count) + "/"+ length, display_id=True)


for rant in rants:

    sentiment = sentiment_analysis(rant["rant"], client)
    rant.update(sentiment)
    sentiments.append(rant)

    count+=1
    info.update(str(count) + "/"+ length)



'13/13'

In [ ]:
len(sentiments)

13

In [ ]:

data= pd.DataFrame(sentiments)

old_data= pd.read_csv("/content/drive/MyDrive/au.rants.23/sentiments.csv")
old_data["date"]= pd.to_datetime(old_data.date)
old_data= old_data[SINCE > old_data.date]

new_data = pd.concat([data, old_data], ignore_index=True)

new_data.to_csv("/content/drive/MyDrive/au.rants.23/sentiments.csv")
new_data

,id,date,rant,positive,neutral,negative,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1
0,2021-08-10_18-16-37,2021-08-10,Can someone going to campus please share how ...,0.04,0.12,0.84,NaN,NaN,NaN
1,2021-08-10_18-13-48,2021-08-10,Do you guys have free mobile game recs?,0.99,0.01,0.00,NaN,NaN,NaN
2,2021-08-09_18-52-29,2021-08-09,WHY ARE ALL OF US NOT COMING TO CAMPUS ?? I W...,0.09,0.84,0.07,NaN,NaN,NaN
3,2021-08-07_17-20-00,2021-08-07,I think I have feelings for my best friend an...,0.98,0.00,0.02,NaN,NaN,NaN
4,2021-08-09_18-53-53,2021-08-09,"My friend's being very stupid about this guy,...",0.50,0.00,0.50,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1422,2021-05-23_06-40-54,2021-05-23 00:00:00,Quite unsettled with CASH. Not seen the campu...,0.00,0.02,0.98,1422.0,1409.0,1238.0
1423,2021-05-23_06-42-42,2021-05-23 00:00:00,Should we be mentally preparing for another s...,0.16,0.46,0.38,1423.0,1410.0,1239.0
1424,2021-05-23_06-44-00,2021-05-23 00:00:00,does anyone else likeo sea shanties,0.00,0.99,0.01,1424.0,1411.0,1240.0
1425,2021-05-23_06-45-26,2021-05-23 00:00:00,Does anyone else feel that CBSE and state boa...,0.06,0.55,0.39,1425.0,1412.0,1241.0


In [ ]:
data

,id,date,rant,positive,neutral,negative
0,2021-08-10_18-16-37,2021-08-10,Can someone going to campus please share how ...,0.04,0.12,0.84
1,2021-08-10_18-13-48,2021-08-10,Do you guys have free mobile game recs?,0.99,0.01,0.00
2,2021-08-09_18-52-29,2021-08-09,WHY ARE ALL OF US NOT COMING TO CAMPUS ?? I W...,0.09,0.84,0.07
3,2021-08-07_17-20-00,2021-08-07,I think I have feelings for my best friend an...,0.98,0.00,0.02
4,2021-08-09_18-53-53,2021-08-09,"My friend's being very stupid about this guy,...",0.50,0.00,0.50
5,2021-08-09_18-55-55,2021-08-09,aurants I can't understand why admin sucks an...,0.38,0.11,0.51
6,2021-08-10_18-15-58,2021-08-10,"Honest question, why are ppl so hyped up bout...",0.64,0.05,0.31
7,2021-08-07_17-51-59,2021-08-07,ARE THERE ACADEMIC COUNSELLORS YOU COULD GET ...,0.41,0.56,0.03
8,2021-08-10_18-15-05,2021-08-10,My friends think he's ugly and I sort of get ...,0.46,0.04,0.50
9,2021-08-09_18-54-37,2021-08-09,tag your fav ashoka friends in the comments a...,0.93,0.06,0.01


## Opnion Mining

In [ ]:
def opinion_mining(id, date, rant, client):

    documents = [rant]

    if type(rant) ==type(""):
        if len(rant) ==0:
            return  None
    else:
        return None
    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    positive_reviews = [doc for doc in doc_result if doc.sentiment == "positive"]
    negative_reviews = [doc for doc in doc_result if doc.sentiment == "negative"]

    positive_mined_opinions = []
    mixed_mined_opinions = []
    negative_mined_opinions = []

    for document in doc_result:
        
        for sentence in document.sentences:
          
            for mined_opinion in sentence.mined_opinions:
                target = mined_opinion.target

                for assessment in mined_opinion.assessments:
                    opnion= {
                        "rant_id": id,
                        "date":date, 
                        "target_text": target.text,
                        "target_sentiment":target.sentiment,
                        "assesment_text":assessment.text,
                        "assesment_sentiment": assessment.sentiment,

                        "target_positive_score": target.confidence_scores.positive,
                        "target_negative_score":target.confidence_scores.negative,
                        "assesment_positive_score": assessment.confidence_scores.positive,
                        "assesment_negative_score": assessment.confidence_scores.negative
                    }

                    return opnion


opnions=[]

count=0
length= str(len(data))
info = display("Compleated: "+str(count) + "/"+ length, display_id=True)   
          
for (rant, id, date) in data.loc[: , ["rant", "id", "date"]].values:
    
    opnion = opinion_mining(id, date, rant, client)
    if opnion != None:
        opnions.append(opnion)
    count+=1
    info.update(str(count) + "/"+ length)   

    
opnions = list(filter(None, opnions))


'13/13'

In [ ]:



old_opnions= pd.read_csv("/content/drive/MyDrive/au.rants.23/opnions.csv")
old_opnions["date"]= pd.to_datetime(old_opnions.date)
old_opnions= old_opnions[SINCE > old_opnions.date]

new_opnions = pd.concat([pd.DataFrame(opnions), old_opnions], ignore_index=True)

new_opnions.to_csv("/content/drive/MyDrive/au.rants.23/opnions.csv")
new_opnions

,rant_id,date,target_text,target_sentiment,assesment_text,assesment_sentiment,target_positive_score,target_negative_score,assesment_positive_score,assesment_negative_score,Unnamed: 0,id
0,2021-08-10_18-12-26,2021-08-10,sounds,negative,stupid,negative,0.01,0.99,0.01,0.99,NaN,NaN
1,NaN,2021-07-30 00:00:00,job,positive,great,positive,1.00,0.00,1.00,0.00,0.0,2021-07-30_15-18-10
2,NaN,2021-07-28 00:00:00,topic,negative,off,negative,0.08,0.92,0.08,0.92,1.0,2021-07-28_16-09-27
3,NaN,2021-07-28 00:00:00,systems,negative,misogynistic,negative,0.01,0.99,0.01,0.99,2.0,2021-07-28_05-21-05
4,NaN,2021-07-23 00:00:00,question,positive,genuine,positive,1.00,0.00,1.00,0.00,3.0,2021-07-23_08-43-47
...,...,...,...,...,...,...,...,...,...,...,...,...
194,NaN,2021-05-22 00:00:00,memes,negative,match,negative,0.06,0.94,0.06,0.94,193.0,2021-05-22_16-39-56
195,NaN,2021-05-22 00:00:00,Professor,negative,failed,negative,0.01,0.99,0.01,0.99,194.0,2021-05-22_16-45-16
196,NaN,2021-05-23 00:00:00,nipun,positive,cute,positive,1.00,0.00,1.00,0.00,195.0,2021-05-23_06-35-35
197,NaN,2021-05-23 00:00:00,place,negative,toxic,negative,0.01,0.99,0.01,0.99,196.0,2021-05-23_06-36-20


## Likes and comments

In [ ]:
likes=[]

count=0
length= str(len(new_data))
info = display("Compleated: "+str(count) + "/"+ length, display_id=True)  

for post in posts:
    id = str(post.date_utc).replace(" ", "_")
    id = id.replace(":", "-")
    likes.append(dict(id = id, date=post.date.date(), likes = post.likes, comments=post.comments))
    
    count+=1
    info.update("Compleated: "+str(count) + "/"+ length)  

likes= pd.DataFrame(likes)


'Compleated: 1426/1427'

In [ ]:
likes.to_csv("/content/drive/MyDrive/au.rants.23/likes_and_comments.csv")
